# Análisis temporal
`
Autores:
Brigitte Aguilar, Sofía Poux, Elizabeth Young
`

Modificado de *PracticalMEEG2022: MNE-python hands-on tutorial*. Por Britta Westner. 

## Setup

Al igual que en el notebook anterior, importamos las librerías a utilizar:

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import mne

mne.set_log_level('error')

## Establece la ruta a los datos y carga las épocas guardadas previamente



Una vez descargada la carpeta `Datos`, tenemos que informarle a Python dónde encontrarla en el disco. ¡Tendrás que ajustar la dirección a continuación!
Podés imprimir la ruta completa y verificar que el directorio sea el correcto.

In [ ]:
# Modificá la siguiente ruta según dónde se encuentre la carpeta Datos en tu disco

data_path = os.path.expanduser("C:\MNE-projects\mne_CuttingGardens_OroVerde\Datos")

epochs_fname = os.path.join(data_path,
    'sub-01\sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif')

In [ ]:
epochs = mne.read_epochs(epochs_fname, proj=True)

In [ ]:
epochs.info

¿Las proyecciones están activadas?


## Explora las diferentes condiciones en los datos

Hemos visto anteriormente que los datos tienen diferentes eventos que marcan qué estímulo se mostró a los participantes.

La estructura de los eventos es la siguiente:

- 5, 6, 7: rostros famosos
- 13, 14, 15: rostros no familiares
- 17, 18, 19: rostros mezclados

Hemos creado una época para los datos de acuerdo con estos eventos previamente.

¡Veamos cómo podemos usar estos códigos de evento para trabajar con las condiciones!

Primero miramos cómo se registraron los eventos:

In [ ]:
epochs

Ahora podemos consultar los datos según los ID de eventos:

In [ ]:
epochs['long']

In [ ]:
epochs['scrambled', 'famous']

In [ ]:
epochs['famous']

Ahora podemos, por ejemplo, usar esto para trazar las épocas según la condición:

In [ ]:
%matplotlib qt

epochs['famous'].plot_image(sigma=1.,
                            picks='EEG001' # Selección de un canal particular
                            );



<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
      <li>¿Podes hacer el mismo gráfico para las otras dos condiciones?</li>
    </ul>
</div>

## Promediar las épocas para obtener ERF/ERP

In [ ]:
evoked = epochs.average()
evoked.apply_proj();

In [ ]:
epochs.average?

In [ ]:
evoked.info

¡Miremos nuestro campo evocado y nuestro potencial evocado!

In [ ]:
evoked.plot?

In [ ]:
%matplotlib qt

evoked.plot();

## Separar las condiciones

Miremos el ERP en el canal EEG065 para las tres condiciones.

In [ ]:
# Recuerda cómo podemos indexar las épocas:
for condition in ['famous', 'unfamiliar', 'scrambled']:
    epochs[condition].average().plot(picks='EEG065');

Para manejar más fácilmente nuestro objeto evocado, hagamos una _lista de objetos evocados_ para las diferentes condiciones que nos interesan. ¡Entonces también podemos hacerlo más fácil para nosotros y comparar los ERP en un solo gráfico!

In [ ]:
conditions = ['famous', 'unfamiliar', 'scrambled']
evokeds_list = []
for k, cond in enumerate(conditions):
   evokeds_list.append(epochs[cond].average().apply_proj().crop(-0.5, 1))
   evokeds_list[k].comment = cond  # we want to update the name of the condition for plotting

In [ ]:
%matplotlib qt
mne.viz.plot_compare_evokeds(evokeds_list, picks='EEG065');

Pero ¿y si queremos ver _todos_ los canales?

In [ ]:
%matplotlib qt
mne.viz.plot_compare_evokeds(evokeds_list, picks='eeg', axes='topo');

<div class="alert alert-success">
    <b>EJERCICIO</b>:
      <ul>
        <li>Intenta hacer clic en el gráfico, ¿qué sucede?</li>
        <li>¿Puedes hacer los mismos gráficos para magnetómetros? y gradiómetros?</li>
    </ul>
</div>

## Trazar la desviación estándar entre los ensayos

Puede resultar útil visualizar los datos evocados con intervalos de confianza o desviaciones estándar.
¡Veamos cómo! Para las desviaciones estándar entre _ensayos_, no existe una función MNE-Python lista para usar.
La razón: ¡los objetos evocados ya no almacenan la información para calcular esta métrica (es decir, pruebas únicas)!

Pero no te preocupes: podemos trazar esto nosotros mismos usando matplotlib, ¡y obtener algunos conocimientos sobre matplotlib y funciones de escritura en el camino!

In [ ]:
# Escribamos nuestra propia función para calcular las desviaciones estándar (sd=standard deviation)
# Dado que tenemos que usar esta operación para cada condición, es útil escribir una función para ella:
#
# Para la función, necesitamos el objeto epochs y queremos elegir este objeto para la condición
# y canal correctos agregamos como argumentos la condición y 'picks'.

def compute_sd(epochs, condition, picks):

    # .get_data devuelve un arreglo (type = numpy) con los datos para la condición y canales seleccionados:
    epochs_data = epochs[condition].get_data(picks=picks)

    # np.mean devuelve la media:
    epochs_mean = np.mean(epochs_data, axis=0)

    # np.std devuelve el desvío estandar:
    epochs_std = np.std(epochs_data, axis=0)

    # se agrega 'arriba' y 'abajo' de la media:
    sd_upper = np.squeeze(epochs_mean + epochs_std)
    sd_lower = np.squeeze(epochs_mean - epochs_std)

    # y devuelve el desvío estandar inferior y superior
    return sd_lower, sd_upper


In [ ]:
# ¡Hemos recortado los evocados, por lo que también tenemos que recortar las épocas

epochs_crop = epochs.copy().crop(-0.5, 1)

In [ ]:
%matplotlib inline

picks = 'EEG065'  # graficamos un canal

# primero agreguemos las líneas en cero que tiene MNE-Python

plt.axhline(y=0., color='black', linewidth=0.75)
plt.axvline(x=0., color='black', linewidth=0.75, linestyle='--')

for ii, condition in enumerate(conditions):

    # obtenemos los valores de desvío estandar con nuestra función
    sd_lower, sd_upper = compute_sd(epochs_crop, condition, picks)

    # obtenemos los datos con get_data
    evoked_data = evokeds_list[ii].get_data(picks=picks)
    times = evokeds_list[ii].times

    # graficamos los datos evocados
    plt.plot(times, np.squeeze(evoked_data), linewidth=2, label=condition)

    # graficamos el desvío estandar y asignamos una etiqueta
    plt.fill_between(times, sd_lower, sd_upper, alpha=0.3,
                     label=('standard dev.' if ii==2 else None))



# etiquetas y leyenda
plt.ylabel('V') # ¡tenga en cuenta la diferencia en la escala a MNE-Python!
plt.xlabel('Time (s)')
plt.legend(loc='lower left', framealpha=0.3)

plt.show()


## Trazar topografías
A continuación, queremos trazar las topografías para todos los tipos de canales _y_ todas las condiciones. Para no terminar con 9 parcelas individuales, ¡veamos cómo combinar parcelas en una sola figura!

In [ ]:
%matplotlib inline
# por tipo de canal, tenemos diferentes límites de color para la topografía:
# color_lims = [(-150, 150), (0, 45), (-4.5, 4.5)]
# channel_types = ['mag', 'grad', 'eeg']
# sup_titles = ['Magnetometers', 'Gradiometers', 'EEG']

# Graficamos solamente para EEG (en caso de graficar para los demas canales, descomentar las lineas de arriba
# y comentar estas)
color_lims = [(-4.5, 4.5)]
channel_types = ['eeg']
sup_titles = ['EEG']


# primero recorremos los tipos de canales:
for ch_type, c_lim, s_title in zip(channel_types, color_lims, sup_titles):

    # Aquí construimos nuestra figura. ¡Ten en cuenta que pedimos 4 subtramas!
    # Esto se debe a que nuestra barra de colores necesita su propio eje.

    fig, axes = plt.subplots(1, 4, figsize=(7, 4), gridspec_kw={'width_ratios': [3, 3, 3, 0.5]})

    # Ahora podemos obtener la hora pico en las condiciones para este tipo de canal.
    # Buscamos el tiempo pico hasta 150 ms después del inicio del estímulo. Trazaremos
    # la topografía durante 100 ms alrededor de este pico; consulte el parámetro promedio en la
    # llamada a la función de trazado.
    _, peak_time = evoked.get_peak(ch_type=ch_type, tmax=0.15)

    for ii, (ax, cond) in enumerate(zip(axes, conditions)):
        evokeds_list[ii].plot_topomap(
            ch_type=ch_type, times=peak_time, average=0.1,
            vlim=c_lim,
            colorbar=(True if ii==2 else False),  # solo una barra de colores
            axes=(axes[2::] if ii == 2 else ax),  # La última trama necesita 2 ejes.
            show=False)

        ax.set_title(cond)

    fig.suptitle('%s, %.3f s' % (s_title, peak_time))

## Escribir datos evocados en el disco

In [ ]:
# tenga en cuenta que el archivo para evocados termina con -ave.fif
evoked_fname = epochs_fname.replace('-epo.fif', '-ave.fif')
evoked_fname

In [ ]:
evoked.save(evoked_fname, overwrite=True)

También puedes escribir varias condiciones en un archivo:

In [ ]:
evoked_list_fname = evoked_fname.replace('-ave.fif', '_list-ave.fif')
mne.write_evokeds(evoked_list_fname, evokeds_list, overwrite=True)

### Lectura evocada desde el disco

Puede volver a leer los datos desde el archivo `.fif` almacenado:

In [ ]:
evokeds_list = mne.read_evokeds(evoked_fname, baseline=(None, 0), proj=True)

O proporcione el nombre explícito de la condición promediada al leer la lista:

In [ ]:
evoked_famous = mne.read_evokeds(evoked_list_fname, condition="famous",
                           baseline=(None, 0), proj=True)
evoked_famous

**Observación:** ¿Notaste que puedes aplicar algún preprocesamiento al leer los evocados desde el disco?

## Contrasta las condiciones

In [ ]:
# ahora podemos usar la condición "face", que consta de famosos y desconocidos
evoked_face = epochs['face'].average().apply_proj()
evoked_scrambled = epochs['scrambled'].average().apply_proj()

# adaptar el nombre
evoked_face.comment = 'face'
evoked_scrambled.comment = 'scrambled'

In [ ]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])

contrast.comment = 'face - scrambled'

Tenga en cuenta que esto combina evocaciones teniendo en cuenta el número de épocas promediadas (para escalar la variación del ruido).

Veamos la cantidad de épocas que promediamos:

In [ ]:
print(evoked_face.nave)
print(evoked_scrambled.nave)
print(contrast.nave)

In [ ]:
print(contrast)

Tracemos el contraste junto con los ERP:

In [ ]:
%matplotlib qt

mne.viz.plot_compare_evokeds([evoked_face, evoked_scrambled, contrast], picks='EEG065');

<div class="alert alert-success">
    <b>EJERCICIO</b>:
     <ul>
      <li>¿Puedes trazar la diferencia entre rostros famosos y desconocidos para otro sensor EEG?</li>
    </ul>
</div>
